In [1]:
import yaml
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

C:\Users\chandan.bhardwaj\AppData\Local\Temp\ipykernel_1704\440697630.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# Load configuration from YAML file
with open('configTesting15022024.yaml', 'r') as file:
    config = yaml.safe_load(file)

proxies = {'http': config['proxies']['http']}

In [3]:
def find_year(date_str):
    """Parse date string and adjust the year."""
    today = datetime.today()
    month, day = map(int, date_str.split('/'))
    year = today.year if month <= today.month else today.year - 1
    return datetime(year=year, month=month, day=day).strftime('%Y-%m-%d')

In [5]:
def fetch_commodity_data(website_config):
    """Fetch and process commodity data from a given website configuration."""
    try:
        response = requests.get(website_config['url'], proxies=proxies)
        response.raise_for_status()  # This will raise an exception for HTTP error codes
        soup = BeautifulSoup(response.content, 'html.parser')
        
        data_rows = []
        parent = soup.find("ul", class_='zwd_table no_select').findAll("li")
        for ele in parent:
            row_data = [p.get_text() for p in ele.findAll("p")]
            if row_data:  # Ensure row_data is not empty
                data_rows.append(row_data)
        
        if data_rows:
            df = pd.DataFrame(data_rows[1:], columns=data_rows[0])
            df['Date'] = df['Date'].apply(find_year)
            df['Price'] = df['Price'].astype(float)
            df['Product Name'] = website_config['product_name']
            df['Type'] = website_config['type']
            df['Currency'] = website_config['currency']
            df['WebsiteName'] = website_config['WebsiteName']
            df['QueryTimestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            df['TableFetchTimestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            return df
            
    except Exception as e:
        print(f"Failed to fetch and process data for {website_config['product_name']}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure

def run():
    all_data_frames = []  # List to store all DataFrames
    for website in config['websites']:
        df = fetch_commodity_data(website)
        if not df.empty:
            all_data_frames.append(df)

    # Concatenate all DataFrames into one
    if all_data_frames:
        combined_df = pd.concat(all_data_frames, ignore_index=True)
        # Save the combined DataFrame to a single CSV file
        combined_csv_filename = "combined_commodities_data.csv"
        combined_df.to_csv(combined_csv_filename, index=False)
        print(f"All data saved to {combined_csv_filename}.")
    else:
        print("No data fetched.")

if __name__ == "__main__":
    run()

All data saved to combined_commodities_data.csv.


In [21]:
# this is still work in progress!
#def fetch_commodity_data(website_config):
    """Fetches and processes commodity data based on website configuration."""
    try:
        response = requests.get(website_config['url'], proxies=proxies)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        data_rows = []
        # Placeholder for actual data extraction logic
        parent = soup.find("ul", class_='zwd_table no_select').findAll("li")
        for ele in parent:
            row_data = [p.get_text() for p in ele.findAll("p")]
            if row_data:
                data_rows.append(row_data)
        
        if data_rows:
            df = pd.DataFrame(data_rows[1:], columns=['Date', 'Price', 'Open', 'High', 'Low', 'Volume', 'ChgPerc'])
            df['Date'] = df['Date'].apply(lambda x: find_year(x).strftime('%Y-%m-%d %H:%M:%S'))
            df['ProductName'] = website_config['product_name']
            df['Type'] = website_config['type']
            df['Currency'] = website_config['currency']
            df['WebsiteName'] = website_config['WebsiteName']
            df['QueryTimestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            df['TableFetchTimestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            # Placeholder for USD_INR and RMB_INR conversion rates, adjust as necessary
            df['USD_INR'] = 0  # Placeholder value
            df['RMB_INR'] = 0  # Placeholder value
            # Generating IdentifierKey
            df['IdentifierKey'] = df.apply(lambda row: f"{row['TableFetchTimestamp']}_{row['ProductName']}_{row['WebsiteName']}", axis=1)
            
            return df
            
    except Exception as e:
        print(f"Failed to fetch and process data for {website_config['product_name']}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of failure

In [26]:
def run():
    combined_data = []
    for website in config['websites']:
        df = fetch_commodity_data(website)
        if not df.empty:
            combined_data.append(df)

    if combined_data:
        combined_df = pd.concat(combined_data, ignore_index=True)
        combined_csv_filename = "combined_commodities_data.csv"
        combined_df.to_csv(combined_csv_filename, index=False)
        print(f"Combined data saved to {combined_csv_filename}.")
    else:
        print("No data fetched.")

In [28]:
if __name__ == "__main__":
    run()

Combined data saved to combined_commodities_data.csv.


In [10]:
def fetch_usd_inr_conversion_rate():
    url = "https://in.investing.com/currencies/usd-inr-historical-data"
    response = requests.get(url, proxies=proxies)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # You need to replace 'your-row-selector' with the actual selector that targets the rate
    rows = soup.select('table.common-table.medium.js-table tr')
    if rows:
        latest_row = rows[1]  # Assuming the first row after the header has the latest rates
        rate = latest_row.select_one('td:nth-child(2)').text  # Assuming the second column has the rate
        return float(rate.replace(',', ''))
    return None


In [11]:

def fetch_rmb_inr_conversion_rate():
    url = "https://in.investing.com/currencies/cny-inr-historical-data"
    response = requests.get(url, proxies=proxies)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    rows = soup.select('table.common-table.medium.js-table tr')
    if rows:
        latest_row = rows[1]
        rate = latest_row.select_one('td:nth-child(2)').text
        return float(rate.replace(',', ''))
    return None


In [29]:
def fetch_commodity_data(website_config,usd_inr_rate, rmb_inr_rate):
    """Fetch and process commodity data based on website configuration."""
    try:
        response = requests.get(website_config['url'], proxies=proxies)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the data table, assuming 'li' elements under 'ul.zwd_table.no_select' contain the data rows
        table_rows = soup.select('ul.zwd_table.no_select li')[1:]  # Skip the first row if it contains headers
        
        data_rows = []
        for row in table_rows:
            # Extract the text from each 'p' tag and strip it
            columns = [col.get_text(strip=True) for col in row.find_all('p')]
            # Now columns[0] should correspond to the 'Date' and columns[1] to the 'Price'
            if len(columns) >= 2:  # Ensure there's at least Date and Price
                date, price = columns[0], columns[1]
                # Here you might need to convert date and price to the appropriate format
                data_rows.append([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # QueryTimestamp
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # TableFetchTimestamp
                    price,  # Price from the page
                    website_config['currency'],  # Currency from config
                    'N/A', 'N/A', 'N/A', 'N/A', 'N/A',  # Other values as N/A or placeholders
                    website_config['product_name'],  # ProductName from config
                    website_config['type'],  # Type from config
                    website_config['WebsiteName'],  # WebsiteName from config
                    'N/A', 'N/A'  # USD_INR and RMB_INR as placeholders
                ])
        
        # Construct DataFrame
        df = pd.DataFrame(data_rows, columns=[
            'QueryTimestamp', 'TableFetchTimestamp', 'Price', 'Currency', 'Open_', 'High', 'Low', 
            'Volume', 'ChgPerc', 'ProductName', 'Type', 'WebsiteName', 'USD_INR', 'RMB_INR'
        ])
        
        # Assume that IdentifierKey is a combination of Timestamp, ProductName, and WebsiteName
        df['IdentifierKey'] = df.apply(
            lambda x: f"{x['TableFetchTimestamp']}_{x['ProductName']}_{x['WebsiteName']}", axis=1
        )
        df['USD_INR'] = usd_inr_rate
        df['RMB_INR'] = rmb_inr_rate
        return df
    
    except Exception as e:
        print(f"An error occurred while fetching data for {website_config['product_name']}: {e}")
        return pd.DataFrame(columns=[
            'QueryTimestamp', 'TableFetchTimestamp', 'Price', 'Currency', 'Open_', 'High', 'Low', 
            'Volume', 'ChgPerc', 'ProductName', 'Type', 'WebsiteName', 'USD_INR', 'RMB_INR', 'IdentifierKey'
        ])


In [30]:
def run():
    usd_inr_rate = fetch_usd_inr_conversion_rate()
    rmb_inr_rate = fetch_rmb_inr_conversion_rate()
    combined_data = []
    for website in config['websites']:
        df = fetch_commodity_data(website,usd_inr_rate, rmb_inr_rate)
        if not df.empty:
            combined_data.append(df)

    if combined_data:
        combined_df = pd.concat(combined_data, ignore_index=True)
        combined_csv_filename = "combined_commodities_data.csv"
        combined_df.to_csv(combined_csv_filename, index=False)
        print(f"Combined data saved to {combined_csv_filename}.")
    else:
        print("No data fetched.")

In [31]:
if __name__ == "__main__":
    run()

Combined data saved to combined_commodities_data.csv.


In [16]:
def fetch_commodity_data(website_config, usd_inr_rate, rmb_inr_rate):
    """Fetch and process commodity data based on website configuration."""
    try:
        response = requests.get(website_config['url'], headers={'User-Agent': 'Mozilla/5.0'}, proxies=proxies)
        soup = BeautifulSoup(response.content, 'html.parser')

        data_rows = []
        table_rows = soup.select('ul.zwd_table.no_select li')[1:]  # Skip the first row if it contains headers
        for row in table_rows:
            columns = [col.get_text(strip=True) for col in row.find_all('p')]
            if len(columns) >= 2:
                date, price = columns[0], columns[1]
                data_rows.append([
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # QueryTimestamp
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # TableFetchTimestamp
                    price,  # Price from the page
                    website_config['currency'],  # Currency from config
                    'N/A', 'N/A', 'N/A', 'N/A', 'N/A',  # Other values as N/A or placeholders
                    website_config['product_name'],  # ProductName from config
                    website_config['type'],  # Type from config
                    website_config['WebsiteName'],  # WebsiteName from config
                    usd_inr_rate,  # USD_INR rate from fetched data
                    rmb_inr_rate,  # RMB_INR rate from fetched data
                ])

        # Construct DataFrame
        df = pd.DataFrame(data_rows, columns=[
            'QueryTimestamp', 'TableFetchTimestamp', 'Price', 'Currency', 'Open_', 'High', 'Low',
            'Volume', 'ChgPerc', 'ProductName', 'Type', 'WebsiteName', 'USD_INR', 'RMB_INR'
        ])

        # Generate IdentifierKey
        df['IdentifierKey'] = df.apply(
            lambda x: f"{x['TableFetchTimestamp']}_{x['ProductName']}_{x['WebsiteName']}", axis=1
        )

        return df
    
    except Exception as e:
        print(f"An error occurred while fetching data for {website_config['product_name']}: {e}")
        return pd.DataFrame(columns=[
            'QueryTimestamp', 'TableFetchTimestamp', 'Price', 'Currency', 'Open_', 'High', 'Low',
            'Volume', 'ChgPerc', 'ProductName', 'Type', 'WebsiteName', 'USD_INR', 'RMB_INR', 'IdentifierKey'
        ])


In [20]:

def fetch_conversion_rate(url, selector):
    """Fetch the latest conversion rate from a given URL."""
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, proxies=proxies)
        soup = BeautifulSoup(response.content, 'html.parser')
        rate_cell = soup.select_one(selector)
        if rate_cell:
            rate_text = rate_cell.get_text(strip=True).replace(',', '')
            return float(rate_text)
        else:
            print(f"No element found for the selector {selector} at URL {url}")
            return None
    except Exception as e:
        print(f"An error occurred while fetching the conversion rate from {url}: {e}")
        return None

def run():
    # Define the selectors for the conversion rates on the Investing.com website
    usd_inr_selector = 'table.common-table.medium.js-table tr:nth-of-type(2) td:nth-of-type(2)'
    rmb_inr_selector = 'table.common-table.medium.js-table tr:nth-of-type(2) td:nth-of-type(2)'
    
    # Fetch the conversion rates using the defined selectors
    usd_inr_rate = fetch_conversion_rate("https://in.investing.com/currencies/usd-inr-historical-data", usd_inr_selector)
    rmb_inr_rate = fetch_conversion_rate("https://in.investing.com/currencies/cny-inr-historical-data", rmb_inr_selector)

    # Make sure the rates were fetched successfully, otherwise set them to None or handle as needed
    if usd_inr_rate is None or rmb_inr_rate is None:
        print("Failed to fetch conversion rates. Exiting.")
        return  # or you can set a default value or handle this case as needed

    combined_data = []
    for website in config['websites']:
        df = fetch_commodity_data(website, usd_inr_rate, rmb_inr_rate)
        if not df.empty:
            combined_data.append(df)

    # Concatenate all DataFrames into one if any data was fetched
    if combined_data:
        combined_df = pd.concat(combined_data, ignore_index=True)
        combined_csv_filename = "combined_commodities_data.csv"
        combined_df.to_csv(combined_csv_filename, index=False)
        print(f"Combined data saved to {combined_csv_filename}.")
    else:
        print("No data fetched.")

# The entry point of the script
if __name__ == "__main__":
    run()


No element found for the selector table.common-table.medium.js-table tr:nth-of-type(2) td:nth-of-type(2) at URL https://in.investing.com/currencies/usd-inr-historical-data
No element found for the selector table.common-table.medium.js-table tr:nth-of-type(2) td:nth-of-type(2) at URL https://in.investing.com/currencies/cny-inr-historical-data
Failed to fetch conversion rates. Exiting.


In [18]:
def fetch_conversion_rate(url, selector):
    """Fetch the latest conversion rate from a given URL."""
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, proxies=proxies)
        soup = BeautifulSoup(response.content, 'html.parser')
        rate_cell = soup.select_one(selector)
        if rate_cell:
            rate_text = rate_cell.get_text(strip=True).replace(',', '')
            return float(rate_text)
        else:
            print(f"No element found for the selector {selector} at URL {url}")
            return None
    except Exception as e:
        print(f"An error occurred while fetching the conversion rate from {url}: {e}")
        return None
